# Narupa ASE Client / Server Example

This notebook demonstrates an example of running an interactive molecular dynamics Narupa server with ASE, connecting to it from a client, looking at the data being produced and visualising it.

## Set up an ASE simulation

First, we set up an ASE simulation. We're going to do a simple lattice under an EMT potential, but you can use [any supported calculator](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html#module-ase.calculators)!

In [20]:
from ase import units
from ase.calculators.emt import EMT
from ase.lattice.cubic import FaceCenteredCubic
from ase.md import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

In [21]:
size = 2
# Set up a crystal
atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          symbol="Cu",
                          size=(size, size, size),
                          pbc=True)
# Describe the interatomic interactions with the Effective Medium Theory
atoms.set_calculator(EMT())
# Set the momenta corresponding to T=50K
MaxwellBoltzmannDistribution(atoms, 300 * units.kB)

Set up our molecular dynamics simulation:

In [22]:
dyn = Langevin(atoms, 1 * units.fs, 300 * units.kB, 0.5)

Let's check it's working:

In [23]:
dyn.run(steps = 1)
dyn.get_number_of_steps()

1

In [24]:
dyn.atoms.get_potential_energy()

-0.18051242903255726

## Set up the Narupa iMD server

We fetch the Narupa object we need to serve the dynamics. We provide a handy wrapper that understands ASE simulations and will serve the dynamics for you.

In [25]:
from narupa.app import NarupaImdApplication
from narupa.ase import NarupaASEDynamics

In [26]:
app_server = NarupaImdApplication.basic_server(port=0) #Try using Shift+TAB+TAB to see what you can set up here.

A server is now ready to be discovered, we can see where it's running with the following:

In [27]:
print(f'{app_server.name}: Running at {app_server.address}:{app_server.port}')

mike: Narupa iMD Server: Running at [::]:55469


The `[::]` means it is running on all available addresses, e.g. your WiFi and cabled access, and on port `38801`.

Now, let's attach it to the ASE simulation

In [28]:
narupa_imd = NarupaASEDynamics(app_server, dyn)

Let's run a couple of steps to make sure it's working

In [29]:
narupa_imd.run(10, block=True)

In [30]:
narupa_imd.dynamics.nsteps

11

Ok, it's working, so let's leave it running dynamics in background thread. This is what happens by default when you call `run`

In [31]:
narupa_imd.run()

In [32]:
# print the time to check dynamics is running
print(f'Simulation time: {narupa_imd.dynamics.get_time()}, ({narupa_imd.dynamics.nsteps} steps)')

Simulation time: 1.375177270384969, (14 steps)


That's it, your simulation is running and interactive! If you connect to it from iMD-VR, you'll see something like:

![Narupa ASE EMT](./images/narupa_ase_emt.png)

You can also visualize it live in the notebook, (check out this [example with NGLView](narupa_interactive_visualiser.ipynb))

## Start an IMD client

Let's have a look at the data that's being produced here, by connecting a python client

In [36]:
from narupa.app import NarupaImdClient
client = NarupaImdClient.autoconnect() # Try doing this from another computer on the same network!

Let's grab a frame

In [37]:
frame = client.latest_frame

In [38]:
frame.particle_count # use TAB to see what's available!

32

We can also print out the raw underlying data that is sent to VR

In [39]:
# view the latest frame
client.latest_frame.raw

values {
  key: "chain.count"
  value {
    number_value: 1.0
  }
}
values {
  key: "energy.kinetic"
  value {
    number_value: 122.60460060608636
  }
}
values {
  key: "energy.potential"
  value {
    number_value: 113.91117193248488
  }
}
values {
  key: "particle.count"
  value {
    number_value: 32.0
  }
}
values {
  key: "residue.count"
  value {
    number_value: 1.0
  }
}
arrays {
  key: "bond.pairs"
  value {
    index_values {
    }
  }
}
arrays {
  key: "chain.names"
  value {
    string_values {
      values: "A"
    }
  }
}
arrays {
  key: "particle.elements"
  value {
    index_values {
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29
      values: 29


Now, let's visualize the frame directly with ASE. First we convert the frame back into ASE atoms, and view it. The first frame received by a client always has the topology information

In [40]:
from narupa.ase.converter import frame_data_to_ase
atoms = frame_data_to_ase(client.first_frame, topology=True, positions=False)

We can run this to update the latest positions from the frame (note that Narupa uses **nanometers** for positions. [Why??](http://docs.openmm.org/6.2.0/userguide/theory.html#units))

In [41]:
import numpy as np
# set the positions to match the latest frame data
atoms.set_positions(np.array(client.latest_frame.particle_positions) * 10)

ASE has a cool little 2D visualizer

In [42]:
from ase.visualize import view
view(atoms)

And a 3D visualizer! (Check your browser)

In [ ]:
view(atoms, viewer='x3d')

# Gracefully terminate!!

It is very important to close your servers, otherwise they'll get in the way by blocking ports and your clients may autoconnect to the wrong server!

In [45]:
narupa_imd.close()
app_server.close()

Note that outside of a notebook, you can use `with` statements to let Narupa clean up after itself:

In [ ]:
with NarupaASEDynamics.basic_imd(dyn) as app:
    app.run(20)

# Next Steps

This notebook showed a toy example with ASE. To run bigger molecular mechanics simulations, we use OpenMM. The [nanotube](./openmm_nanotube.ipynb) and [neuraminidase](./openmm_neuraminidase.ipynb) examples show how to set up simulations with OpenMM.

To understand what's going on under the hood, consider starting with the [Narupa frame explained](../fundamentals/frame.ipynb) example. 